In [1]:
import sys
sys.path.insert(0, "../lib")

In [2]:
import math
import os
import pathlib
import re

import scanpy as sc
import numpy as np
import pandas as pd
import sc_utils
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [18]:
pd.options.display.max_columns = None

In [3]:
DATA = pathlib.Path('../../data/31_bal-object/')

In [4]:
adata = sc.read_h5ad(DATA / "03_annotated.h5ad")

Attach last-minute metadata

In [9]:
meta = pd.read_csv('../00all-samples.csv')

In [16]:
for col in meta.columns[~meta.columns.isin(adata.obs.columns)]:
    prev_col = meta.columns.get_loc(col) - 1
    prev_col_name = meta.columns[prev_col]
    idx = adata.obs['External Sample ID']
    sample_to_col = meta.set_index('External Sample ID')[col]
    adata.obs.insert(adata.obs.columns.get_loc(prev_col_name) + 1, col, sample_to_col[idx].values)

In [21]:
OBJ = DATA / "03_bal-object"

In [22]:
os.makedirs(OBJ, exist_ok=True)

In [23]:
adata.obs.to_csv(OBJ / "03_bal-object-metadata.csv")

In [8]:
sc.tl.rank_genes_groups(adata, "cell_type", method="t-test", n_genes=200)

In [9]:
markers = sc_utils.get_markers(adata, "cell_type")

In [10]:
markers.sort_values(["cluster", "avg_logFC"], ascending=[True, False], inplace=True)

In [11]:
markers.to_csv(OBJ / "03_bal-object-markers.csv")

Drop `gene_ids` because cellbrowser uses them automatically, which is not compatible with our markers

In [24]:
adata.var.drop("gene_ids", axis=1, inplace=True)

In [25]:
adata.raw.var.drop("gene_ids", axis=1, inplace=True)

In [26]:
adata.write_h5ad(OBJ / "03_bal-object.h5ad")

... storing 'Protocol' as categorical
